<a href="https://colab.research.google.com/github/AquaFAN171029/Qwen-Fire/blob/main/Qwen_image_testing-updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install diffusers>=0.35.1 transformers accelerate safetensors pillow

import os, torch
from PIL import Image, ImageDraw
from diffusers import QwenImageEditPipeline

from PIL import Image
import numpy as np
import cv2
import os, json

edit = QwenImageEditPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit",
    torch_dtype=torch.bfloat16
    )
edit.enable_model_cpu_offload()
edit.text_encoder.to("cpu")
print("loaded")



base = Image.open("back.png").convert("RGB")
x1,y1,x2,y2 = 799, 643, 935, 962
pad = 12
x1p=max(0,x1-pad); y1p=max(0,y1-pad); x2p=min(base.width,x2+pad); y2p=min(base.height,y2+pad)
roi = base.crop((x1p,y1p,x2p,y2p))

#在 ROI上插入灭火器
prompt = ("Insert a photorealistic handheld fire extinguisher in the central area; "
          "red cylindrical metal body, black hose, metal nozzle, pressure gauge, "
          "generic safety label, soft contact shadow, consistent with surrounding lighting")
out_roi = edit(
    image=roi,
    prompt=prompt,
    negative_prompt="cartoon, painting, logo, watermark, blurry, deformed",
    true_cfg_scale=4.0,
    num_inference_steps=50,
    generator=torch.manual_seed(42)
).images[0]

# 转坐标到int，判定坐标范围
W, H = base.size
x1  = int(x1);  y1  = int(y1);  x2  = int(x2);  y2  = int(y2)
x1p = max(0, int(x1p)); y1p = max(0, int(y1p))
x2p = min(W, int(x2p)); y2p = min(H, int(y2p))

#以 roi 的真实尺寸，重新推导右下角
rw, rh = roi.size
x2p = x1p + rw
y2p = y1p + rh
assert x2p <= W and y2p <= H, "ROI Error"

#开始贴回：

base_np = np.array(base.convert("RGB"))

#贴回(x1p,y1p,x2p,y2p)
target_w, target_h = (x2p - x1p), (y2p - y1p)
out_roi_resized = out_roi.resize((target_w, target_h), Image.LANCZOS)
out_np = np.array(out_roi_resized.convert("RGB"))
#左上+右下同时对齐
assert out_np.shape[1] == target_w and out_np.shape[0] == target_h
base_np[y1p:y2p, x1p:x2p, :] = out_np


# 保存
os.makedirs("out", exist_ok=True)
composited = Image.fromarray(base_np, mode="RGB")
composited.save("out/out.png")
roi.save("out/roi.png") # 裁下来的原始ROI
out_roi.save("out/roi_ai.png") # AI生成的ROI
print("Saved")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 79.32 GiB of which 19.88 MiB is free. Process 66580 has 79.29 GiB memory in use. Of the allocated memory 78.70 GiB is allocated by PyTorch, and 96.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!pip -q install diffusers>=0.35.1 transformers accelerate safetensors pillow

import os, torch
from PIL import Image, ImageDraw
from diffusers import QwenImageEditPipeline

print(torch.__version__)

2.8.0+cu126


In [ ]:
from PIL import Image
import numpy as np
import cv2
import os, json

edit.enable_model_cpu_offload()

base = Image.open("back.png").convert("RGB")
x1,y1,x2,y2 = 799, 643, 935, 962
pad = 12
x1p=max(0,x1-pad); y1p=max(0,y1-pad); x2p=min(base.width,x2+pad); y2p=min(base.height,y2+pad)
roi = base.crop((x1p,y1p,x2p,y2p))

#在 ROI上插入灭火器
prompt = ("Insert a photorealistic handheld fire extinguisher in the central area; "
          "red cylindrical metal body, black hose, metal nozzle, pressure gauge, "
          "generic safety label, soft contact shadow, consistent with surrounding lighting")
out_roi = edit(
    image=roi,
    prompt=prompt,
    negative_prompt="cartoon, painting, logo, watermark, blurry, deformed",
    true_cfg_scale=4.0,
    num_inference_steps=50,
    generator=torch.manual_seed(42)
).images[0]

# 转坐标到int，判定坐标范围
W, H = base.size
x1  = int(x1);  y1  = int(y1);  x2  = int(x2);  y2  = int(y2)
x1p = max(0, int(x1p)); y1p = max(0, int(y1p))
x2p = min(W, int(x2p)); y2p = min(H, int(y2p))

#以 roi 的真实尺寸，重新推导右下角
rw, rh = roi.size
x2p = x1p + rw
y2p = y1p + rh
assert x2p <= W and y2p <= H, "ROI Error"

#开始贴回：

base_np = np.array(base.convert("RGB"))

#贴回(x1p,y1p,x2p,y2p)
target_w, target_h = (x2p - x1p), (y2p - y1p)
out_roi_resized = out_roi.resize((target_w, target_h), Image.LANCZOS)
out_np = np.array(out_roi_resized.convert("RGB"))
#左上+右下同时对齐
assert out_np.shape[1] == target_w and out_np.shape[0] == target_h
base_np[y1p:y2p, x1p:x2p, :] = out_np


# 保存
os.makedirs("out", exist_ok=True)
composited = Image.fromarray(base_np, mode="RGB")
composited.save("out/out.png")
roi.save("out/roi.png") # 裁下来的原始ROI
out_roi.save("out/roi_ai.png") # AI生成的ROI
print("Saved")

  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 388.00 MiB. GPU 0 has a total capacity of 79.32 GiB of which 259.88 MiB is free. Process 64338 has 79.05 GiB memory in use. Of the allocated memory 77.77 GiB is allocated by PyTorch, and 812.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from diffusers import QwenImageEditPipeline
import torch

use_cuda = torch.cuda.is_available()

# 1) 选择 dtype（GPU 用 fp16；CPU 用 fp32）
dtype = torch.float16 if use_cuda else torch.float32

edit = QwenImageEditPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit",
    torch_dtype=dtype,         # 主体用 fp16（或 fp32）
)

# 2) 上到设备
device = "cuda" if use_cuda else "cpu"
edit.to(device)

# 3) 省显存 & 稳定性开关
try:
    edit.enable_xformers_memory_efficient_attention()
except Exception:
    pass
edit.enable_vae_slicing()

# 4) **关键**：VAE 在 fp16 下容易黑图 → 强制上浮到 fp32
#   方法A（推荐）：让 VAE 需要时自动用 fp32
edit.vae.config.force_upcast = True

from PIL import Image
import numpy as np
import cv2
import os, json

edit.enable_model_cpu_offload()

base = Image.open("back.png").convert("RGB")
x1,y1,x2,y2 = 799, 643, 935, 962
pad = 12
x1p=max(0,x1-pad); y1p=max(0,y1-pad); x2p=min(base.width,x2+pad); y2p=min(base.height,y2+pad)
roi = base.crop((x1p,y1p,x2p,y2p))

#在 ROI上插入灭火器
prompt = ("Insert a photorealistic handheld fire extinguisher in the central area; "
          "red cylindrical metal body, black hose, metal nozzle, pressure gauge, "
          "generic safety label, soft contact shadow, consistent with surrounding lighting")
out_roi = edit(
    image=roi,
    prompt=prompt,
    negative_prompt="cartoon, painting, logo, watermark, blurry, deformed",
    true_cfg_scale=4.0,
    num_inference_steps=50,
    generator=torch.manual_seed(42)
).images[0]

# 转坐标到int，判定坐标范围
W, H = base.size
x1  = int(x1);  y1  = int(y1);  x2  = int(x2);  y2  = int(y2)
x1p = max(0, int(x1p)); y1p = max(0, int(y1p))
x2p = min(W, int(x2p)); y2p = min(H, int(y2p))

#以 roi 的真实尺寸，重新推导右下角
rw, rh = roi.size
x2p = x1p + rw
y2p = y1p + rh
assert x2p <= W and y2p <= H, "ROI Error"

#开始贴回：

base_np = np.array(base.convert("RGB"))

#贴回(x1p,y1p,x2p,y2p)
target_w, target_h = (x2p - x1p), (y2p - y1p)
out_roi_resized = out_roi.resize((target_w, target_h), Image.LANCZOS)
out_np = np.array(out_roi_resized.convert("RGB"))
#左上+右下同时对齐（四点约束）
assert out_np.shape[1] == target_w and out_np.shape[0] == target_h
base_np[y1p:y2p, x1p:x2p, :] = out_np


# 保存
os.makedirs("out2", exist_ok=True)
composited = Image.fromarray(base_np, mode="RGB")
composited.save("out2/out.png")
roi.save("out2/roi.png") # 裁下来的原始ROI
out_roi.save("out2/roi_ai.png") # AI生成的ROI
print("Saved")



Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipython-input-3851312893.py:82: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  composited = Image.fromarray(base_np, mode="RGB")


Saved
